Description:
This notebook merges data from covid-19 open data and Nomis 

In [1]:
#importing necessary modules
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

#plotting
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import cufflinks as cf
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import os

# Merging all datasets into one

## Google mobility

In [ ]:
#downloading the data
geography = pd.read_csv(r"https://storage.googleapis.com/covid19-open-data/v2/geography.csv")
geography.to_pickle(r'data/geography.pkl')

mobility = pd.read_csv(r"https://storage.googleapis.com/covid19-open-data/v2/mobility.csv")
mobility.to_pickle(r'data/mobility.pkl')

weather = pd.read_csv(r"https://storage.googleapis.com/covid19-open-data/v2/weather.csv")
weather.to_pickle(r'data/weather.pkl')

In [7]:
def clean_df(df_name, feature_columns):
    '''
    function to clean csv file and select the columns used in the analysis
    '''
    df = pd.read_pickle(r'data/'+df_name)

    df = df[(pd.notnull(df['key']))&(df['key'].str.contains('GB_ENG'))].copy()

    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['date']>=pd.to_datetime('2020-03-25'))&(\
                df['date']<=pd.to_datetime('2020-04-25'))].copy()
    df = df.groupby(['key'])[feature_columns].mean()
    return df.reset_index()

In [8]:
df_mobility = clean_df(r'mobility.pkl',['mobility_retail_and_recreation',
       'mobility_grocery_and_pharmacy', 'mobility_parks',
       'mobility_transit_stations', 'mobility_workplaces',
       'mobility_residential'])
df_weather = clean_df(r'weather.pkl',['average_temperature','rainfall'])

df_geography = pd.read_pickle(r'data/geography.pkl')
df_geography = df_geography[(pd.notnull(df_geography['key']))&(df_geography['key'].str.contains('GB_ENG'))].copy()
df_geography = df_geography[['key','latitude','longitude','area']].copy()
df_geography = df_geography.rename(columns = {'area':'surface'})

In [9]:
df_google = pd.merge(df_mobility,df_weather, on = 'key')
df_google = pd.merge(df_google,df_geography, on = 'key')

## Workforce by sector and Age structure from Nomis

### Workforce

In [10]:
#Nomis
xl = pd.ExcelFile(r'data/nomis_2021_01_01_232934.xlsx')
df = xl.parse('Data', skiprows=7, usecols=(0,2,3,7,11,15,19,23,27,31,35,39))
df.columns = ['area','workforce','managers','professional','associate','administrative','skilled_trades',
             'caring','sales','process','elementary']
df = df[pd.notnull(df['area'])].copy()
df = df[df['area'].str.contains('lacu:')].copy()
df['area'] = df['area'].apply(lambda x: str(x).split(':')[1])
df = df.set_index('area')
df = df.apply(pd.to_numeric, errors='coerce')
df_workforce = df.reset_index()

### Age Distribution

In [11]:
xl = pd.ExcelFile(r'data/nomis_2021_01_02_080844.xlsx')
df = xl.parse('Data',skiprows=8)
column_names = [x for x in df.columns[1:] if 'Unnamed' not in x]
df = xl.parse('Data',skiprows=9)
df = df[[x for x in df.columns if 'number' not in x]].copy()
df = df.iloc[:185].copy()
df.columns = ['area','area_code']+column_names
df = df.set_index(['area','area_code'])
df = df.apply(pd.to_numeric, errors='coerce')
df_age = df.reset_index()
df_age = df_age[pd.notnull(df_age['All usual residents'])].copy()
df_age['key']  = df_age['area_code'].apply(lambda x: 'GB_ENG_'+x)

df_age['share_65'] = df_age[['Age 65 to 74', 'Age 75 to 84', 'Age 85 to 89',
       'Age 90 and over']].sum(axis = 1)

In [12]:
df_population = xl.parse('Data',skiprows=9)
df_population = df_population[[x for x in df_population.columns if ('number' in x) or ('Unnamed' in x)]].copy()
df_population = df_population.iloc[:185].copy()
df_population.columns = ['area','area_code']+column_names
df_population = df_population.rename(columns = {'All usual residents':'population'})
df_age = pd.merge(df_age, df_population[['area','population']], on='area', how='left')

## Merging

In [13]:
df = pd.merge(df_workforce,df_age, on = 'area')
df = pd.merge(df,df_google, on = 'key')
df['population_density'] = df['population']/df['surface']
df = df.dropna()
df.to_pickle(r'data/df.pkl')

In [14]:
df['area'].nunique()

103

In [16]:
df.shape[0]

103